In [1]:
import sys
sys.path.append('/home/jochemsoons/FACT_COURSE/FACT')

In [ ]:
!wget https://nlp.stanford.edu/projects/snli/snli_1.0.zip
!unzip snli_1.0.zip

--2021-01-08 15:44:46--  https://nlp.stanford.edu/projects/snli/snli_1.0.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 94550081 (90M) [application/zip]
Saving to: ‘snli_1.0.zip’

snli_1.0.zip        100%[===================>]  90.17M  4.36MB/s    in 27s     

2021-01-08 15:45:13 (3.37 MB/s) - ‘snli_1.0.zip’ saved [94550081/94550081]

Archive:  snli_1.0.zip
replace snli_1.0/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
keys = ['train', 'test', 'dev']

In [4]:
import json

In [5]:
data = {k:[] for k in keys}
for k in keys :
    for line in open('snli_1.0/snli_1.0_' + k + '.jsonl').readlines() :
        data[k].append(json.loads(line))

In [6]:
from Transparency.preprocess.vectorizer import cleaner
from tqdm import tqdm_notebook

In [7]:
p, q, a = {}, {}, {}

for k in keys :
    p[k] = [cleaner(x['sentence1']) for x in tqdm_notebook(data[k]) if x['gold_label'] != '-']
    q[k] = [cleaner(x['sentence2']) for x in tqdm_notebook(data[k]) if x['gold_label'] != '-']
    a[k] = [cleaner(x['gold_label']) for x in tqdm_notebook(data[k]) if x['gold_label'] != '-']

In [8]:
entity_list = ['neutral', 'contradiction', 'entailment']
f = open('entity_list.txt', 'w')
f.write("\n".join(entity_list))
f.close()

In [9]:
import pandas as pd
df_paragraphs = []
df_questions = []
df_answers = []
df_exp_splits = []

for k in keys :
    df_paragraphs += p[k]
    df_questions += q[k]
    df_answers += a[k]
    df_exp_splits += [k] * len(p[k])
    
df = {'paragraph' : df_paragraphs, 'question' : df_questions, 'answer' : df_answers, 'exp_split' : df_exp_splits}
df = pd.DataFrame(df)

In [10]:
df.to_csv('snli_dataset.csv', index=False)

In [11]:
%run "../preprocess_data_QA.py" --data_file snli_dataset.csv --output_file ./vec_snli.p --all_answers_file entity_list.txt \
--word_vectors_type glove.840B.300d --min_df 3

Vocabulary size :  20981
entity2index {'contradiction': 0, 'entailment': 1, 'neutral': 2}
Found 20339 words in model out of 20981
